Load dataset:

In [1]:
from HumanGenomeDataset.load_dataset import load_dataset

data_df = load_dataset('dna_protein_coding_sequences')
data_df.head(2)

,ID,sequence,category
0,NM_001368254.1,GCTGAGCTGAGCTGGGGCGCAGCCGCCTGTCTGCACCGGCAGCACC...,NM
1,NM_001350977.1,TCCACAACTGAAACATCCACTTCTGAACACCATGTCCTACTACAGC...,NM


Start simulation:

In [2]:
from src.simulation import ProteinSinthesisProcess
SIM_TIME = 30

In [3]:
protein_synthesis_process = ProteinSinthesisProcess(dna_sequences_df=data_df, verbose=True)
protein_synthesis_process.run(simulation_time=SIM_TIME) # run the simulation

Simulation environment initialized 	
Simulation started: 

Time 0: Protein synthesis started for sequence 0
Time 0: Transcription started
Time 1: Transcription ended
mRNA synthesized: 6
Time 1: Translation started
Time 1: Translation ended
Protein synthesized: 6 

Time 7.7334049334357: Protein synthesis started for sequence 1
Time 7.7334049334357: Transcription started
Time 7.7334049334357: Transcription ended
Protein synthesized: 0 

Time 8.06936925800192: Protein synthesis started for sequence 2
Time 8.06936925800192: Transcription started
Time 9.06936925800192: Transcription ended
mRNA synthesized: 16
Time 9.06936925800192: Translation started
Time 9.06936925800192: Translation ended
Protein synthesized: 16 

Time 13.475901335783856: Protein synthesis started for sequence 3
Time 13.475901335783856: Transcription started
Time 14.475901335783856: Transcription ended
mRNA synthesized: 21
Time 14.475901335783856: Translation started
Time 14.475901335783856: Translation ended
Protein syn

Results:

In [4]:
results_df = protein_synthesis_process.dna_sequences_df

In [5]:
print('Number of DNA sequence traslated:', results_df[results_df['mrna_sequences'].notna()].shape[0])
print('Number of DNA sequence not traslated:', results_df[results_df['protein_synthesized']==False].shape[0])

Number of DNA sequence traslated: 4
Number of DNA sequence not traslated: 3


In [6]:
results_df[results_df['mrna_sequences'].notna()][[
    'polypeptides_chains', 'number_of_proteins_synthesized']]

,polypeptides_chains,number_of_proteins_synthesized
95673,"[NH2-VSCYLWTYYK-COOH, NH2-CLFKCKFSHDMNKKIL-COOH]",2
149563,"[NH2-KNKIFPVVMYSTIKYKVHVNTRSGNTISLK-COOH, NH2-...",21
167621,"[NH2-YIGIGKK-COOH, NH2-V-COOH, NH2-YVVPIKKTHKN...",6
185125,"[NH2--COOH, NH2-NLNEQVRYLILYRRYVPFF-COOH, NH2-...",16
